# Getting Started...

In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import graphviz
import pydotplus
import io

from scipy import misc

from sklearn.metrics import accuracy_score

import spotipy
import spotipy.util as util
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 

%matplotlib inline

# Spotify Login

In [2]:
cid ="YOUR_CLIENT_ID" 
secret = "YOUR_CLIENT_SECRET"
username = "USERNAME"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri='LOCATION')

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

# Input Data into the Model

In [13]:
good_playlist = sp.user_playlist("jongill0", "spotify:playlist:6BxoDn1x1AqUKP1xtRl9Dg")
bad_playlist = sp.user_playlist("jongill0", "spotify:playlist:3Rt5UnbIXg77Y6DRGNRsGg")

In [14]:
#Get the good song id's out of the good playlists, need it for the audio features call.
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"] 
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)
good_ids = [] 
for i in range(len(good_songs)- 500):
    good_ids.append(good_songs[i]['track']['id'])
    

#Now do the same thing for the bad playlist
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"] 
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)
bad_ids = [] 
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])

1454
871


In [15]:
#Here is where we get all of the audio_features for the tracks on the good and bad playlists.
features = []
inSavedTracks = []
j = 0
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs[j]
        j= j+1
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['artistPopularity'] = sp.artist(track['track']['artists'][0]['id'])['popularity']
        features[-1]['target'] = 1
j = 0
for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs[j]
        j= j+1
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['artistPopularity'] = sp.artist(track['track']['artists'][0]['id'])['popularity']
        features[-1]['target'] = 0

In [16]:
trainingData = pd.DataFrame(features)
trainingData.head()

,acousticness,analysis_url,artistPopularity,danceability,duration_ms,energy,id,instrumentalness,key,liveness,...,mode,speechiness,target,tempo,time_signature,trackPopularity,track_href,type,uri,valence
0,0.00372,https://api.spotify.com/v1/audio-analysis/6Gat...,70,0.675,300729,0.500,6GatnDEhYQLOdRzdVbJaky,0.002680,1,0.114,...,1,0.0516,1,90.029,4,60,https://api.spotify.com/v1/tracks/6GatnDEhYQLO...,audio_features,spotify:track:6GatnDEhYQLOdRzdVbJaky,0.0709
1,0.10900,https://api.spotify.com/v1/audio-analysis/1JUv...,70,0.624,248432,0.536,1JUvB8LRKYoik4BB1bEJWK,0.000428,2,0.102,...,0,0.0799,1,119.088,4,54,https://api.spotify.com/v1/tracks/1JUvB8LRKYoi...,audio_features,spotify:track:1JUvB8LRKYoik4BB1bEJWK,0.1310
2,0.14600,https://api.spotify.com/v1/audio-analysis/47wR...,70,0.517,308178,0.488,47wRhI5SQeEq6YXawou58W,0.064300,4,0.101,...,0,0.0903,1,74.787,4,44,https://api.spotify.com/v1/tracks/47wRhI5SQeEq...,audio_features,spotify:track:47wRhI5SQeEq6YXawou58W,0.0578
3,0.00368,https://api.spotify.com/v1/audio-analysis/50aq...,70,0.718,214583,0.640,50aqcJ4eeth6yKNg4pAP1D,0.000042,5,0.334,...,1,0.0591,1,110.007,4,47,https://api.spotify.com/v1/tracks/50aqcJ4eeth6...,audio_features,spotify:track:50aqcJ4eeth6yKNg4pAP1D,0.1890
4,0.40900,https://api.spotify.com/v1/audio-analysis/6ddq...,48,0.372,228535,0.413,6ddqBHpdgaM8nWYZZDdTqP,0.528000,4,0.106,...,0,0.0379,1,126.607,3,32,https://api.spotify.com/v1/tracks/6ddqBHpdgaM8...,audio_features,spotify:track:6ddqBHpdgaM8nWYZZDdTqP,0.1600


In [51]:
train, test = train_test_split(trainingData, test_size = 0.20)
print("Training size: {}, Test size: {}".format(len(train),len(test)))

Training size: 1460, Test size: 365


In [52]:
#Define the set of features that we want to look at
features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness","duration_ms"]

In [53]:
#Split the data into x and y test and train sets to feed them into a bunch of classifiers!
x_train = train[features]
y_train = train["target"]

x_test = test[features]
y_test = test["target"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
forest.fit(x_train, y_train)
forest_pred = forest.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, forest_pred) * 100
print("Accuracy: ", round(score, 2), "%")

In [88]:
# Request to modify playlists
username = "jongill0"
scope = 'playlist-modify-private playlist-modify-public playlist-read-private user-library-read'
token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri='LOCATION')
#Good Playlist
if token:
    sp = spotipy.Spotify(auth=token)

In [89]:
playlistToFindSongsYouLikeIn = sp.user_playlist("Spotify", "spotify:playlist:37i9dQZF1DX2Nc3B70tvx0")

newPlaylist_tracks = playlistToFindSongsYouLikeIn["tracks"]
newPlaylist_songs = newPlaylist_tracks["items"] 
while newPlaylist_tracks['next']:
    newPlaylist_tracks = sp.next(newPlaylist_tracks)
    for song in newPlaylist_tracks["items"]:
        newPlaylist_songs.append(song)
        
newPlaylist_song_ids = [] 
print(len(newPlaylist_songs))
for i in range(len(newPlaylist_songs)):
    newPlaylist_song_ids.append(newPlaylist_songs[i]['track']['id'])
    
newPlaylist_features = []
j = 0
for i in range(0,len(newPlaylist_song_ids),50):
    audio_features = sp.audio_features(newPlaylist_song_ids[i:i+50])
    for track in audio_features:
        track['song_title'] = newPlaylist_songs[j]['track']['name']
        track['artist'] = newPlaylist_songs[j]['track']['artists'][0]['name']
        j= j + 1
        newPlaylist_features.append(track)
print(len(newPlaylist_features))

playlistToLookAtFeatures = pd.DataFrame(newPlaylist_features)

80
80


In [117]:
pred = forest.predict(playlistToLookAtFeatures[features])

likedSongs = 0
i = 0

for prediction in pred:
    if (prediction == 1):
        print("Song: " + playlistToLookAtFeatures["song_title"][i] + ", By: "+ playlistToLookAtFeatures["artist"][i])
        likedSongs= likedSongs + 1
    i = i +1
print(likedSongs)

Song: Hey, Ma, By: Bon Iver
Song: Some Place Else, By: MorMor
Song: Bruise, By: Yumi Zouma
Song: Patience, By: Tame Impala
Song: Someone To Spend Time With, By: Los Retros
Song: He, By: Jai Paul
Song: Bags, By: Clairo
Song: Love Yourself, By: Sufjan Stevens
Song: Are You OK?, By: Wasuremono
Song: Lava, By: Still Woozy
Song: The Barrel, By: Aldous Harding
Song: Hey You, By: DOPE LEMON
Song: Ghostride, By: Crumb
Song: A Song for You, By: Kishi Bashi
Song: No Halo, By: Kevin Morby
Song: Daytrip, By: Stephan Looney
Song: Took A Fall, By: Golden Daze
Song: Now That You're Gone, By: The Raconteurs
Song: The Bottom of It, By: Fruit Bats
Song: Turn The Light, By: Karen O
Song: Geology, By: Barrie
Song: Flowers, By: Ra Ra Riot
Song: Weekend Friend, By: Goth Babe
Song: Manifest, By: Andrew Bird
Song: 50 Souls and a Discobowl - Recorded at Spotify Studios Stockholm, By: boy pablo
Song: Harmony Hall, By: Vampire Weekend
Song: Sunlight, By: Yuno
Song: Only Friend, By: Wallows
Song: Borderline, By: 